In [1]:

import torch

from editing_diffusion.llm_objects import spot_objects
from editing_diffusion.detectors.owlvitv2 import OWLViTv2Detector

device = torch.device("cuda:1")

/home/andrey/anaconda3/envs/editing_diffusion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model and tokenizer names
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


In [3]:
model_params = {
    "max_length": 1200,  # Maximum length of generated text
    #"temperature": 0.7,  # Controls randomness (higher -> more random)
    #"top_k": 50, 
}

In [4]:
prompts = [
    "a realistic cartoon-style image with a princess and four dwarfs",
    "a vivid photo with a woman on the right and a clown on the left walking in a dirty alley",
    "a monkey sitting above a green motorcycle on the left and another raccoon sitting above a blue motorcycle on the right",
    "a photo of a giant macaron and a croissant splashing in the Seine with the Eiffel Tower in the background",
    "a DSLR photo of a meatball and a donut falling from the clouds onto a neighborhood",
]
results = []
for prompt in prompts:
    result = spot_objects(tokenizer, model, prompt, device, **model_params)
    results.append(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

In [5]:
results

[{'objects': [('princess', [None]), ('dwarf', [None, None, None, None])],
  'bg_prompt': 'A realistic cartoon-style image',
  'neg_prompt': ''},
 {'objects': [('woman', [None]), ('clown', [None])],
  'bg_prompt': 'A dirty alley',
  'neg_prompt': ''},
 {'objects': [('monkey', ['sitting']),
   ('motorcycle', ['green']),
   ('raccoon', ['sitting']),
   ('motorcycle', ['blue'])],
  'bg_prompt': 'A realistic image',
  'neg_prompt': ''},
 {'objects': [('macaron', ['giant']),
   ('croissant', ['splashing']),
   ('Eiffel Tower', [None])],
  'bg_prompt': 'A photo of a giant macaron and a croissant splashing in the Seine with the Eiffel Tower in the background',
  'neg_prompt': ''},
 {'objects': [('meatball', [None]),
   ('donut', [None]),
   ('clouds', [None]),
   ('neighborhood', [None])],
  'bg_prompt': 'A DSLR photo',
  'neg_prompt': ''}]

In [6]:
results[0]

{'objects': [('princess', [None]), ('dwarf', [None, None, None, None])],
 'bg_prompt': 'A realistic cartoon-style image',
 'neg_prompt': ''}

In [7]:
detector = OWLViTv2Detector(device)

In [8]:
detector.register_objects(results[0]["objects"])

In [9]:
detector.primitive_count, detector.attribute_count

({'princess': 1, 'dwarf': 4}, {})

In [10]:
detector.pred_primitive_count, detector.pred_attribute_count

({'princess': 0, 'dwarf': 0}, {})